In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
from time import sleep
import random
import pandas as pd

In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import requests

In [5]:
# 바이너리 오류가 발생할 경우
# 크롬 바이너리 경로와 크롬 드라이버 바이너리 경로를 둘다 명시해줘야함
options = webdriver.ChromeOptions()
options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
chrome_driver_binary = "./chromedriver"
driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
print("##########driver open##########")

/var/folders/rg/mxvp06m56_g09wl1jz84qkhh0000gn/T/ipykernel_74844/699548518.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
/var/folders/rg/mxvp06m56_g09wl1jz84qkhh0000gn/T/ipykernel_74844/699548518.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)


##########driver open##########


In [6]:
wait = WebDriverWait(driver, 600)

# 크롤링

## url 접속 및 df 열 정의

In [7]:
site = "http://ie.hanyang.ac.kr/index.php?pg=1&page=list&hCode=BOARD&bo_idx=1&sfl=&stx="

driver.get(url = site)
sleep(random.randint(1, 3))

data = {
        '번호': [],
        '분류': [],
        '등록시간': [],
        '작성자' : [],
        '제목': [],
        '본문': [],
        'url': [],
        '첨부파일': [],
       }

## 공지 수 
## 한 페이지의 최대 콘텐츠 수
## 총 콘텐츠 수

In [8]:
# 페이지의 콘텐츠 번호 리스트
content_num = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/table/tbody/tr/td[1]")
print("첫 페이지의 콘텐츠 수 :", len(content_num))

# 공지의 수
announce_num = 0
for i in range(len(content_num)):
    if content_num[i].text == "공지":
        announce_num += 1
    else:
        break
print("공지의 개수 :", announce_num)

max_content_num = len(content_num)-announce_num
print("한 페이지의 공지를 제외한 최대 콘텐츠의 개수:", max_content_num)

# 총 콘텐츠의 수
num_of_content = content_num[announce_num].text
print("공지를 제외한 총 콘텐츠의 개수 :", num_of_content)

첫 페이지의 콘텐츠 수 : 41
공지의 개수 : 21
한 페이지의 공지를 제외한 최대 콘텐츠의 개수: 20
공지를 제외한 총 콘텐츠의 개수 : 738


## 공지 크롤링

- 콘텐츠 크롤링과 함께 묶지 않은 이유 : 그렇게 할 경우 매 페이지마다 공지를 중복으로 크롤링함

In [9]:
for i in range(len(content_num)):
    
    # 페이지 로드 대기
    sleep(random.randint(1, 3))
    
    # stale 오류 때문에 위의 변수를 계속 사용하지 못하고 계속 업데이트 해야함
    content_num = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/table/tbody/tr/td[1]")
    
    if content_num[i].text == "공지":
        print("--------------------------------------------")
        
        data['번호'].append("공지")
        print('번호 :', "공지")
        data['분류'].append("산업공학과")
        print('분류 :', "산업공학과")
        
        # 글 클릭
        con_button = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/table/tbody/tr[" + str(i+1) + "]/td[2]/a")[0]
        con_button.click()
        
        # 등록시간 수집
        upload_time = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[2]/td/ul/li[5]")[0]
        upload_time = upload_time.text
        data['등록시간'].append(upload_time)
        print('등록시간 :', upload_time)
        
        # 작성자 수집
        writer = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[2]/td/ul/li[1]")[0]
        writer = writer.text
        data['작성자'].append(writer)
        print('작성자 :', writer)
        
        # 제목 수집
        title = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[1]/th")[0]
        title = title.text
        data['제목'].append(title)
        print('제목 :', title)

        # 본문 수집
        main = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/tbody/tr/td")[0]
        main = main.get_attribute('innerHTML')
        data['본문'].append(main)
        print('본문 :', main[:50], '~')
        
        # url 수집
        data['url'].append(driver.current_url)
        print('url :',driver.current_url)
        
        # 첨부파일 수집
        try:
            file_button = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[3]/td/ul/li/a")[0]
            file_button.click()
            
            files = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[3]/td/ul/li/div/ul")
            file_urls = []
            for i in range(len(files)):
                file = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[3]/td/ul/li/div/ul["+str(i+1)+"]/li/span/a")[0]
                file = file.get_attribute('href')
                file_urls.append(file)
            
            data['첨부파일'].append(file_urls)
            print('첨부파일 :', file_urls)
            
            driver.back()
        except:
            empty_list = []
            data['첨부파일'].append(empty_list)
            print('첨부파일 : 없음')
        
        # 뒤로가기
        driver.back()
    else:
        break
        

--------------------------------------------
번호 : 공지
분류 : 산업공학과
등록시간 : 2023-08-03 13:03:52
작성자 : 관리자
제목 : [학부]음악대학 마이크로전공 신설 및 모집 안내
본문 : 
						<div class="board_content">
							<img src ~
url : http://ie.hanyang.ac.kr/index.php?page=view&pg=1&idx=2044&hCode=BOARD&bo_idx=1&sfl=&stx=
첨부파일 : 없음
--------------------------------------------
번호 : 공지
분류 : 산업공학과
등록시간 : 2023-07-26 12:20:14
작성자 : 관리자
제목 : [학부]2023학년도 2학기 군 e-러닝 과정 안내
본문 : 
						<div class="board_content">
							<span st ~
url : http://ie.hanyang.ac.kr/index.php?page=view&pg=1&idx=2034&hCode=BOARD&bo_idx=1&sfl=&stx=
첨부파일 : 없음
--------------------------------------------
번호 : 공지
분류 : 산업공학과
등록시간 : 2023-07-26 12:07:16
작성자 : 관리자
제목 : 2023학년도 2학기 및 (연계)겨울학기 현장실습 운영 안내
본문 : 
						<div class="board_content">
							<img src ~
url : http://ie.hanyang.ac.kr/index.php?page=view&pg=1&idx=2033&hCode=BOARD&bo_idx=1&sfl=&stx=
첨부파일 : 없음
--------------------------------------------
번호 : 공지
분류 : 산업공학과
등록시간 : 2023-07-25 13:50:06
작성자 : 관리자
제목

## 공지 제외 콘텐츠 크롤링

In [10]:
for i in range((int(num_of_content)//max_content_num)+1):
    
    # 마지막 페이지의 콘텐츠 수가 다를 수 있으므로
    content_num = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/table/tbody/tr/td[1]")
    
    for i in range(len(content_num)):

        # 페이지 로드 대기
        sleep(random.randint(1, 3))

        # stale 오류 때문에 위의 변수를 계속 사용하지 못하고 계속 업데이트 해야함
        content_num = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/table/tbody/tr/td[1]")

        if content_num[i].text != "공지":
            print("--------------------------------------------")
            
            data['번호'].append(content_num[i].text)
            print('번호 :', content_num[i].text)
            data['분류'].append("산업공학과")
            print('분류 :', "산업공학과")

            # 글 클릭
            con_button = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/table/tbody/tr[" + str(i+1) + "]/td[2]/a")[0]
            con_button.click()

            # 등록시간 수집
            upload_time = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[2]/td/ul/li[5]")[0]
            upload_time = upload_time.text
            
            # 2023년 이전 자료는 크롤링 하지 않음
            if int(upload_time[:4]) < 2023:
                driver.back()
                break
                
            # 작성자 수집
            writer = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[2]/td/ul/li[1]")[0]
            writer = writer.text
            data['작성자'].append(writer)
            print('작성자 :', writer)    
                
            data['등록시간'].append(upload_time)
            print('등록시간 :', upload_time)

            # 제목 수집
            title = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[1]/th")[0]
            title = title.text
            data['제목'].append(title)
            print('제목 :', title)

            # 본문 수집
            main = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/tbody/tr/td")[0]
            main = main.get_attribute('innerHTML')
            data['본문'].append(main)
            print('본문 :', main[:50], '~')

            # url 수집
            data['url'].append(driver.current_url)
            print('url :',driver.current_url)
            
            # 첨부파일 수집
            try:
                file_button = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[3]/td/ul/li/a")[0]
                file_button.click()

                files = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[3]/td/ul/li/div/ul")
                file_urls = []
                for i in range(len(files)):
                    file = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[3]/td/ul/li/div/ul["+str(i+1)+"]/li/span/a")[0]
                    file = file.get_attribute('href')
                    file_urls.append(file)

                data['첨부파일'].append(file_urls)
                print('첨부파일 :', file_urls)

                driver.back()
            except:
                empty_list = []
                data['첨부파일'].append(empty_list)
                print('첨부파일 : 없음')

            # 뒤로가기
            driver.back()
            
        else:
            continue
            
    next_button = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/div/ul/a[11]/img")[0]
    next_button.click()
    
    # 2023년 이전 자료는 크롤링 하지 않음
    if int(upload_time[:4]) < 2023:
        data['번호'].pop()
        data['분류'].pop()
        break


--------------------------------------------
번호 : 738
분류 : 산업공학과
작성자 : 관리자
등록시간 : 2023-07-24 12:42:15
제목 : [학부]2022학년도 후기 학위수여식 공로상 대상자 신청 안내
본문 : 
						<div class="board_content">
							<pre sty ~
url : http://ie.hanyang.ac.kr/index.php?page=view&pg=1&idx=2020&hCode=BOARD&bo_idx=1&sfl=&stx=
첨부파일 : 없음
--------------------------------------------
번호 : 737
분류 : 산업공학과
작성자 : 관리자
등록시간 : 2023-06-22 06:52:43
제목 : 2023년 8월 졸업예정자의「학사학위취득유예」및「다전공포기」신청 안내
본문 : 
						<div class="board_content">
							<pre sty ~
url : http://ie.hanyang.ac.kr/index.php?page=view&pg=1&idx=1993&hCode=BOARD&bo_idx=1&sfl=&stx=
첨부파일 : 없음
--------------------------------------------
번호 : 736
분류 : 산업공학과
작성자 : 관리자
등록시간 : 2023-06-15 11:47:47
제목 : 2023학년도 2학기 학생생활관 정규입사 모집일정 안내
본문 : 
						<div class="board_content">
							<pre sty ~
url : http://ie.hanyang.ac.kr/index.php?page=view&pg=1&idx=1987&hCode=BOARD&bo_idx=1&sfl=&stx=
첨부파일 : ['http://ie.hanyang.ac.kr/NFUpload/nfupload_down.php?tmp_name=1503129889_01da788b.pdf&na

In [11]:
# min_length = min(len(data[key]) for key in data)

# for key in data:
#     data[key] = data[key][:min_length]

In [12]:
df = pd.DataFrame(data)

df.tail(8)

,번호,분류,등록시간,작성자,제목,본문,url,첨부파일
48,711,산업공학과,2023-02-01 14:17:32,관리자,2022학년도 졸업앨범 및 졸업액자 교내 배부 안내,"\n\t\t\t\t\t\t<div class=""board_content"">\n\t\...",http://ie.hanyang.ac.kr/index.php?page=view&pg...,[http://ie.hanyang.ac.kr/NFUpload/nfupload_dow...
49,710,산업공학과,2023-02-01 14:13:35,관리자,2023년 1학기 신입생 영어기초학력평가 안내,"\n\t\t\t\t\t\t<div class=""board_content"">\n\t\...",http://ie.hanyang.ac.kr/index.php?page=view&pg...,[http://ie.hanyang.ac.kr/NFUpload/nfupload_dow...
50,709,산업공학과,2023-02-01 13:45:24,관리자,[학부]2023-1학기 수강신청 일정 안내 및 산업공학과 시간표 관련 안내,"\n\t\t\t\t\t\t<div class=""board_content"">\n\t\...",http://ie.hanyang.ac.kr/index.php?page=view&pg...,[http://ie.hanyang.ac.kr/NFUpload/nfupload_dow...
51,708,산업공학과,2023-01-30 09:08:25,관리자,[학부]2023학년도 1학기 사회봉사교과목 운영 안내,"\n\t\t\t\t\t\t<div class=""board_content"">\n\t\...",http://ie.hanyang.ac.kr/index.php?page=view&pg...,[]
52,707,산업공학과,2023-01-25 09:11:57,관리자,2023학년도 1학기 재학생 등록안내,"\n\t\t\t\t\t\t<div class=""board_content"">\n\t\...",http://ie.hanyang.ac.kr/index.php?page=view&pg...,[http://ie.hanyang.ac.kr/NFUpload/nfupload_dow...
53,706,산업공학과,2023-01-13 09:10:04,관리자,[학부]2022학년도 전기 학위수여식 공로상 대상자 신청 안내,"\n\t\t\t\t\t\t<div class=""board_content"">\n\t\...",http://ie.hanyang.ac.kr/index.php?page=view&pg...,[]
54,705,산업공학과,2023-01-11 11:59:31,관리자,[학부]2023-1학기 공과대학 교내 장학금(가계곤란장학) 추가신청 안내,"\n\t\t\t\t\t\t<div class=""board_content"">\n\t\...",http://ie.hanyang.ac.kr/index.php?page=view&pg...,[]
55,704,산업공학과,2023-01-06 07:56:13,관리자,2023학년도 1학기 국내대학 학점교류학생 추천 안내,"\n\t\t\t\t\t\t<div class=""board_content"">\n\t\...",http://ie.hanyang.ac.kr/index.php?page=view&pg...,[http://ie.hanyang.ac.kr/NFUpload/nfupload_dow...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   번호      56 non-null     object
 1   분류      56 non-null     object
 2   등록시간    56 non-null     object
 3   작성자     56 non-null     object
 4   제목      56 non-null     object
 5   본문      56 non-null     object
 6   url     56 non-null     object
 7   첨부파일    56 non-null     object
dtypes: object(8)
memory usage: 3.6+ KB


In [14]:
driver.close()

In [15]:
df.to_csv('hy_noti_ie.csv', index=False)